# eq05-1.txt

- 5桁のランダムな自然数の偶奇を判定するタスク
- 奇数のとき"O"(Odd), 偶数の時"E"(Even)として変換


In [4]:
import random

# 10桁の自然数を生成
def generate_number():
    return str(random.randint(10**4, 10**5 - 1))

# 偶数か奇数かを判定
def is_even(number):
    return "E" if int(number[-1]) % 2 == 0 else "O"

# データセット用に10桁の自然数と偶奇を生成
def generate_number_and_label():
    number = generate_number()  # 10桁の自然数を生成
    label = is_even(number)  # 偶数か奇数かを判定
    return f"{number} => {label}"

# ファイルに保存する関数
def save_to_file(num_numbers, filename=None):
    if filename is None:
        for _ in range(num_numbers):
            print(generate_number_and_label())
        return
    with open(filename, 'w') as file:
        for _ in range(num_numbers):
            data = generate_number_and_label()
            file.write(data + '\n')

# 例として10個の自然数を生成して判定
save_to_file(10)


19195 => O
87199 => O
20180 => E
10937 => O
16992 => E
27872 => E
59512 => E
84502 => E
37509 => O
15418 => E


In [5]:
save_to_file(10000, 'data/eq05-1.txt')
save_to_file(5000, 'data/eq05-1-test.txt')

# eq05-2.txt

- 5桁（2進数）のランダムな自然数を反転させるタスク

In [1]:
import random

# 5桁の2進数を生成
def generate_binary_number():
    return f"{random.randint(0, 31):05b}"  # 0〜31を5桁の2進数としてフォーマット

# データセット用に5桁の2進数とその反転を生成
def generate_binary_number_and_label():
    binary_number = generate_binary_number()  # 5桁の2進数を生成
    label = binary_number[::-1]  # 2進数を反転
    return f"{binary_number} => {label}"

# ファイルに保存する関数
def save_to_file(num_numbers, filename=None):
    if filename is None:
        for _ in range(num_numbers):
            print(generate_binary_number_and_label())
        return
    with open(filename, 'w') as file:
        for _ in range(num_numbers):
            data = generate_binary_number_and_label()
            file.write(data + '\n')

# 例として10個の2進数を生成して反転
save_to_file(10)


10001 => 10001
00101 => 10100
10100 => 00101
01000 => 00010
00000 => 00000
11000 => 00011
01001 => 10010
00010 => 01000
10001 => 10001
00101 => 10100


In [2]:
save_to_file(10000, 'data/eq05-2.txt')
save_to_file(5000, 'data/eq05-2-test.txt')

# eq05-3.txt

- 中置記法ポーランド記法に変換
- 演算子は+のみ
- 深さを1-3に固定
- 深さごとに同数だけ生成
- 01のみ使用
- 空白なし

In [2]:
import random
import operator

DEPTH_LIMIT = 3

# 数式生成に使うオペレータと対応するPythonの演算子
operators = {
    '+': operator.add,
    '*': operator.mul,  # '*' を追加
}

# ランダムな数式を生成する関数（中置記法）
def generate_expression(depth=0, max_depth=DEPTH_LIMIT):
    if depth >= max_depth or random.random() < 0.25:
        return str(random.randint(0, 1))  # 数字のみを返す
    else:
        op = random.choice(list(operators.keys()))  # '+' か '*' を選択
        left = generate_expression(depth + 1, max_depth)
        right = generate_expression(depth + 1, max_depth)
        return f"{left}{op}{right}"

# 中置記法から前置記法（ポーランド記法）に変換する関数
def infix_to_prefix(expression):
    def precedence(op):
        # 演算子の優先順位を定義
        if op == '+' or op == '-':
            return 1
        if op == '*' or op == '/':
            return 2
        return 0

    def apply_operator(operators, values):
        # 演算子を適用して結果を作成
        operator = operators.pop()
        right = values.pop()
        left = values.pop()
        values.append(f"{operator} {left} {right}")

    operators = []
    values = []
    i = 0
    while i < len(expression):
        if expression[i] == '(':
            operators.append(expression[i])
        elif expression[i].isdigit():
            val = ""
            while i < len(expression) and expression[i].isdigit():
                val += expression[i]
                i += 1
            values.append(val)
            continue
        elif expression[i] == ')':
            # '(' まで演算子を適用
            while len(operators) != 0 and operators[-1] != '(':
                apply_operator(operators, values)
            operators.pop()  # '(' を削除
        else:
            # 演算子の場合
            while (len(operators) != 0 and
                   precedence(operators[-1]) > precedence(expression[i])):
                apply_operator(operators, values)
            operators.append(expression[i])
        i += 1

    # 残っている演算子を全て適用
    while len(operators) != 0:
        apply_operator(operators, values)

    return values[-1]

# 構文木のノードを表現するクラス
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

# 数式をポーランド記法として扱い、構文木を構築
def parse_polish(expression):
    tokens = expression.split()
    stack = []
    
    for token in reversed(tokens):
        node = Node(token)
        if token in operators:
            node.left = stack.pop()
            node.right = stack.pop()
        stack.append(node)
    
    return stack[0]  # 最終的な構文木

# データセット用に式を生成
def generate_polish_and_reverse_polish(max_depth=DEPTH_LIMIT):
    expr = generate_expression(max_depth=max_depth)  # ランダムな数式生成
    polish_expr = infix_to_prefix(expr)  # 中置記法から前置記法へ変換
    polish_expr = polish_expr.replace(' ', '')  # スペースを削除
    return f"{expr} => {polish_expr}"

# ファイルに保存する関数
def save_to_file(num_equations, filename=None):
    if filename == None:
        for i in range(num_equations):
            print(generate_polish_and_reverse_polish(max_depth=(i % DEPTH_LIMIT+1)))
        return
    with open(filename, 'w') as file:
        for i in range(num_equations):
            equation = generate_polish_and_reverse_polish(max_depth=(i % DEPTH_LIMIT+1))
            file.write(equation + '\n')

# 例として10個の数式を生成
save_to_file(10)

0*0 => *00
0*1+0*1 => +*01*01
1*0*1+0*1 => +*1*01*01
1+1 => +11
0+1+0+1 => +0+1+01
0 => 0
1*1 => *11
1+0*0 => +1*00
0*1*1*0+0 => +*0*1*100
0 => 0


In [3]:
save_to_file(1000000, 'data/eq05-3.txt')
save_to_file(2000, 'data/eq05-3-test.txt')
